# Predicting Like Polynomial Terms

Remember in Algebra how you had to combine like terms to simplify problems? 

You'd see expressions like `60 + 2x^3 - 6x + x^3 + 17x` in which there are **5** total terms but only **4** are "like terms". `2x^3` and `x^3` are like, and `-6x` and `17x` are like, while `60` has like siblings.

Can we teach a model to predict that there are `4` like terms in the above expression?

Let's give it a shot using mathy to generate math problems and thinc to build a regression model that outputs the number of like terms in the input problems.

In [ ]:
!pip install thinc mathy

### Encode Text Inputs

Mathy generates ascii-math problem texts that we need to encode into scalar values for the model to process. We'll one-hot encode the text by individual characters to keep things simple.

For math problems our vocabulary will include all the characters of the alphabet, numbers 0-9, and the special characters used for operators like `*`, `/`, `.`, etc.

Thinc has a `to_categorical` helper that we can use for converting values into one-hot arrays. Let's use that to write a function that accepts an input string and returns a one-hot representation.

In [3]:
from thinc.types import Array2d
from thinc.api import Ops, get_current_ops, to_categorical

vocab = " .+-/^*()[]-01234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"

def encode_input(text: str) -> Array2d:
    ops: Ops = get_current_ops()
    vocab_indices: List[int] = [vocab.index(s) for s in text]
    return to_categorical(ops.asarray(vocab_indices), n_classes=len(vocab))


#### Try It

Let's try it out on some fixed data to be sure it works. 

We should expect to see an output of one-hot arrays, one for each character in the input.

In [26]:
outputs = encode_input("4+2")
assert outputs[0].argmax() == vocab.index("4")
assert outputs[1].argmax() == vocab.index("+")
assert outputs[2].argmax() == vocab.index("2")
print(outputs)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]


### Generate Math Problems

We'll use [Mathy](https://mathy.ai) to generate [random polynomial problems](https://mathy.ai/api/problems/#gen_simplify_multiple_terms) with a variable number of like terms. The generated problems will act as training data for our model.

In [12]:
from typing import List, Optional, Set
import random
from mathy.problems import gen_simplify_multiple_terms


def generate_problems(number: int, exclude: Optional[Set[str]] = None) -> List[str]:
    if exclude is None:
        exclude = set()
    problems: List[str] = []
    while len(problems) < number:
        text, _ = gen_simplify_multiple_terms(
            random.randint(3, 8), noise_probability=1.0, noise_terms=10
        )
        assert text not in exclude, "duplicate problem generated!"
        problems.append(text)
    return problems


#### Try It

In [14]:
generate_problems(10)

['9n + 1d + 7.3b + 1299j + 4.2m + 10.4r + 8576s + 5o + 1.6u + -5815f + 8f + -4808c + 8.2w',
 '3u^2 + 5.8j + 4015g + 6318c + 1o + 1k + 7p + -5447m + -9561h^3 + 1.0m + 1587p + 9t + 8831f^3 + 7m + 4m + 2.2n + 9.8p',
 '6d^4 + 2j + 7198.9t + 7555a + 4.4g^2 + -1301q + 9.1u^4 + 8.6o + 1.7x + 4.7o + -701x + 8695o + 6307x + 0.8o + 12x + 12z^2 + -6021.8h^3 + -3735.5r^3',
 '6j^2 + 2g + 3545.2t + 8.2o^4 + 1806.0r + 10.2w + 3606l^3 + 10h + 8.7p + 1.3k + 8.7y + 6.9k + 613y + 5k + -9546y + 8k + 4y + -9984d^2',
 '5x + -3606v^3 + 4519m^2 + 10a + -1455g + 9z^3 + 6.2s^3 + 9627z^3 + 4887m^2 + -1819j + 6071h^3 + -6923u + -9845z^3 + 1y^4 + -4590m^2 + 2.0q',
 '2.8o + 11l^4 + 2.8q + 6564.7j^3 + 11.3y + -6816s + -8879b + -9585k + -8861c + 3k + 7d + 4626z + 1x^4',
 '-7233w^4 + 1.2v + 6.0t^4 + -3303o + 5j^4 + 4.9z^2 + 2043y^2 + -9716.6z^2 + 2553.5y^2 + 10.0z^2 + 4.3y^2 + 4z^2 + 5y^2 + 6.9m + -1259k^4 + 5596b + -7434g + 7797f^4',
 '4t + 9.3l + -3121.9q^2 + -6156x + 10.3u^4 + 8f + 1420h + 9.0v + 8079.6m + -1533v +

### Count Like Terms

Now that we can generate input problems, we'll need a function that can count the like terms in each one and return the value for use as a label.

To accomplish this we'll use a few helpers from mathy to enumerate the terms and compare them to see if they're like.

In [20]:
from typing import Optional, List, Dict
from mathy import MathExpression, ExpressionParser, get_terms, get_term_ex, TermEx
from mathy.problems import mathy_term_string

parser = ExpressionParser()

def count_like_terms(input_problem: str) -> int:
    expression: MathExpression = parser.parse(input_problem)
    term_nodes: List[MathExpression] = get_terms(expression)
    node_groups: Dict[str, List[int]] = {}
    for term_node in term_nodes:
        ex: Optional[TermEx] = get_term_ex(term_node)
        assert ex is not None, f"invalid expression {ex}"
        key = mathy_term_string(variable=ex.variable, exponent=ex.exponent)
        if key == "":
            key = "const"
        if key not in node_groups:
            node_groups[key] = [term_node]
        else:
            node_groups[key].append(term_node)
    like_terms = 0
    for k, v in node_groups.items():
        if len(v) <= 1:
            continue
        like_terms += len(v)
    return like_terms

#### Try It

In [24]:
assert count_like_terms("4x + 2y + q") == 0
assert count_like_terms("x + x + z") == 2
assert count_like_terms("4x + 2x + x + 7") == 3

### Generate Problem/Answer pairs

Now that we can generate problems, count the number of like terms in them, and encode their text into one-hot vectors, we have the pieces required to generate random problems and answers that we can train a neural network on.

Let's write a function that will return a tuple of: the problem text, its encoded example form, and the output label.

In [28]:
from typing import Tuple
from thinc.types import Array2d

def to_example(input_problem: str) -> Tuple[str, Array2d, int]:
    one_hot = encode_input(input_problem)
    like_terms = count_like_terms(input_problem)
    return input_problem, one_hot, like_terms

#### Try It

In [32]:
text, X, Y = to_example("x+2x")
# text is preserved for debugging/visualization
assert text == "x+2x"
# X contains one-hot vectors
assert X[0].argmax() == vocab.index("x")
# Y is the number of like terms
assert Y == 2
print(text, X, Y)

x+2x [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]] 2


### Build a Model

Now that we can generate X/Y values, let's define our model and verify it can process a single input/output.

For this we'll use Thinc and define a model that takes in an `Array2d` of examples and outputs an `Array1d` of scalar values, one for each example input.

In [37]:
from typing import List
from thinc.model import Model
from thinc.types import Array2d, Array1d
from thinc.api import chain, list2ragged, reduce_sum, ReLu

def CountLikeTerms(
    n_hidden: int, dropout: float = 0.5
) -> Model[List[Array2d], Array1d]:
    with Model.define_operators({">>": chain}):
        model = (
            list2ragged()
            >> reduce_sum()
            >> ReLu(n_hidden, dropout=dropout)
            >> ReLu(n_hidden)
            >> ReLu(n_hidden)
            >> ReLu(n_hidden)
            >> ReLu(1)
        )
    return model

#### Try It

Let's pass an example through the model to make sure we have all the sizes right.

In [54]:
text, X, Y = to_example("14x + 2y - 3x + 7x")
m = CountLikeTerms(12)
m.initialize([X], Y)
assert m.predict([X]).shape == (1, 1)

### Generate Training Datasets

Now that we can generate examples and we have a model that can process them, let's generate random unique training and evaluation datasets.

For this we'll write another helper function that can generate (n) training examples and respects an exclude list to avoid letting examples from the training/test sets overlap.

In [49]:
from typing import Optional, Set, List
from thinc.types import Array2d

def generate_dataset(
    size: int, exclude: Optional[Set[str]] = None
) -> Tuple[List[str], List[Array2d], List[int]]:
    texts: List[str] = generate_problems(size)
    examples: List[Array2d] = []
    labels: List[int] = []
    for i, text in enumerate(texts):
        text, x, y = to_example(text)
        examples.append(x)
        labels.append(y)

    return texts, examples, labels

#### Try It

Generate a small dataset to be sure everything is working as expected

In [51]:
generate_dataset(3)

(['6b + 8.8u^2 + 3d + 9940g + 1234v + 7.3j + 2n + -340l + 9.8n + 11.1l + 1n + 3319f^2 + 4r + 6.8o + 2t',
  '-9599a + 11.4s + 1k^3 + 12j + 1o + 5.1u + 4d + 2266.0y^4 + -9721n^2 + 10y^4 + 7934n^2 + -5319y^4 + 1.2n^2 + 1.4y^4 + -3386x + 7v + 8.8t^2',
  '4784.3z + 9s + 0.7n + -7046y + 7w + 4.0r + -3554l + 9.2x + 3.3j^3 + 8u^3 + 8m + 1u^3 + -234m + 7036u^3 + 11.4m + 6143u^3 + 5034.5m + 1215q^4'],
 [array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., .

### Evaluate Model Performance

We're almost ready to train our model, we just need to write a function that will check a given trained model against a given dataset and return a 0-1 score of how accurate it was.

We'll use this function to print the score as training progresses and print final test predictions at the end of training.

In [67]:
from typing import List
from thinc.model import Model
from thinc.types import Array2d, Array1d
from wasabi import msg


def evaluate_model(
    model: Model[Array2d, Array1d],
    *,
    print_problems: bool = False,
    texts: List[str],
    X: Array2d,
    Y: List[int],
):
    Yeval = model.predict(X)
    correct_count = 0
    print_n = 10
    if print_problems:
        msg.divider(f"eval samples max({print_n})")
    for text, y_answer, y_guess in zip(texts, Y, Yeval):
        y_guess = round(float(y_guess))
        correct = y_guess == int(y_answer)
        print_fn = msg.fail
        if correct:
            correct_count += 1
            print_fn = msg.good
        if print_problems and print_n > 0:
            print_n -= 1
            print_fn(f"Answer[{y_answer}] Guess[{y_guess}] Text: {text}")
    return correct_count / len(X)


#### Try It

Let's try it out with an untrained model and expect to see a really sad score.

In [60]:
texts, X, Y = generate_dataset(128)
m = CountLikeTerms(12)
m.initialize(X, Y)
# Assume the model should do so poorly as to round down to 0
assert round(evaluate_model(m, texts=texts, X=X, Y=Y)) == 0

### Train the Model


In [71]:
from thinc.api import Adam
from wasabi import msg
import numpy

batch_size = 12
num_iters = 12
train_size = 50000
test_size = 128
seen_texts: Set[str] = set()  

# Set the random seed to make results more reproducible
from thinc.api import fix_random_seed
fix_random_seed(1234)

with msg.loading(f"Generating train dataset with {train_size} examples..."):
    (train_texts, train_X, train_y) = generate_dataset(train_size, seen_texts)
msg.good(f"Train set created with {train_size} examples.")
with msg.loading(f"Generating eval dataset with {test_size} examples..."):
    (eval_texts, eval_X, eval_y) = generate_dataset(test_size, seen_texts)
msg.good(f"Eval set created with {test_size} examples.")
model = CountLikeTerms(512)
model.initialize(train_X[:2], train_y[:2])
optimizer = Adam()
for n in range(num_iters):
    loss = 0.0
    batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
    for X, Y in batches:
        Yh, backprop = model.begin_update(X)
        d_loss = []
        for i in range(len(Yh)):
            d_loss.append(Yh[i] - Y[i])
            loss += ((Yh[i] - Y[i]) ** 2).sum()
        backprop(numpy.array(d_loss))
        model.finish_update(optimizer)
    score = evaluate_model(model, texts=eval_texts, X=eval_X, Y=eval_y)
    print(f"{n}\t{score:.2f}\t{loss:.2f}")

Yeval = model.predict(eval_X)
score = evaluate_model(
    model, print_problems=True, texts=eval_texts, X=eval_X, Y=eval_y
)
print(f"Final Accuracy: {score}")

nerating train dataset with 50000 examples...✔ Train set created with 50000 examples.
nerating eval dataset with 128 examples...✔ Eval set created with 128 examples.
0	0.39	120064.43
1	0.45	41674.67
2	0.53	29810.06
3	0.52	23767.92
4	0.55	20037.54
5	0.64	16832.54
6	0.60	14723.80
7	0.66	13340.37
8	0.72	12555.34
9	0.73	12078.98
10	0.73	11292.49
11	0.75	10791.31

============================ eval samples max(10) ============================
✔ Answer[4] Guess[4] Text: 6.9o + -4124g^4 + 4r + 10s^3 + 0.2v + 6d +
-7098q^2 + 7.3g^4 + 4p + 4k + 5975x^4 + 11m^2 + 0.4l^3 + 10.4l^3
✘ Answer[4] Guess[5] Text: 2510q + -8759a + 12x + 9y + 0.5c + 1880m +
10.9t^4 + 1g + 1143.9c + 4.0u^4 + 3.7y + 8356j + 5n^3 + 5.0s
✔ Answer[2] Guess[2] Text: 9f + 1.8w^3 + 11.0j + 7969n^4 + -4606x +
3604h^3 + 5573.4z + -8408u + 4s^4 + 2u + 6v + 10g^2 + 6p^2
✔ Answer[7] Guess[7] Text: 8d + 9t + 6.4n + -7238.6z^2 + 9.0d + 11w +
6m^4 + 2x^4 + 12c + 1022z^2 + 4y + 8335l + 7.6d + 8430f + -6979d + 9004u^2 +
-6079z^2
✔ Answer[4

### Advanced Exercise

Rewrite the model to encode the whole expression with a BiLSTM, and then generate pairs of terms, using the BiLSTM vectors. Over each pair of terms, predict whether the terms are alike or unlike.

In [ ]:
from dataclasses import dataclass
from thinc.types import Array2d, Ragged
from thinc.mode import Model


@dataclass
class Comparisons:
    data: Array2d  # Batch of vectors for each pair
    indices: Array2d  # Integers array of shape (N, 3), showing the (batch, term1, term2) positions

    def pairify(self) -> Model[Ragged, "Comparisons"]:
        """Create pair-wise comparisons for items in a sequence. For each sequence of N
        items, there will be (N**2-N)/2 comparisons."""
        ...

    def predict_over_pairs(
        self, model: Model[Array2d, Array2d]
    ) -> Model["Comparisons", "Comparisons"]:
        """Apply a prediction model over a batch of comparisons. Outputs a Comparisons
        object where the data is the scores. The prediction model should predict over
        two classes, True and False."""
        ...
